In [1]:
from contrans import contrans
import numpy as np
import pandas as pd
import dotenv
import os
import json
import requests 
import psycopg
import sqlite3
import plotly.express as px
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sqlalchemy import create_engine    
dotenv.load_dotenv()
congresskey = os.getenv('congresskey')
postgrespassword = os.getenv('POSTGRES_PASSWORD')

In [2]:
ct = contrans()

In [3]:
server, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD)

In [4]:
myquery = '''
SELECT *
FROM members
'''
pd.read_sql_query(myquery, con=engine)  

,bioguideid,district,name,partyname,state,updatedate,url,depiction_attribution,depiction_imageurl,partyletter,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,N000188,01,"Norcross, Donald",Democratic,NJ,2024-11-05T20:52:08Z,https://api.congress.gov/v3/member/N000188?for...,Image courtesy of the Member,https://www.congress.gov/img/member/n000188_20...,D,...,None,-0.419,0.340,-50.96336,0.95060,1006.0,23.0,None,-0.391,0.370
1,Y000064,S,"Young, Todd",Republican,IN,2024-11-04T13:42:35Z,https://api.congress.gov/v3/member/Y000064?for...,Image courtesy of the Member,https://www.congress.gov/img/member/y000064_20...,R,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,W000802,S,"Whitehouse, Sheldon",Democratic,RI,2024-11-04T13:42:34Z,https://api.congress.gov/v3/member/W000802?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/w000802_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,W000800,S,"Welch, Peter",Democratic,VT,2024-11-04T13:42:34Z,https://api.congress.gov/v3/member/W000800?for...,Image courtesy of the Member,https://www.congress.gov/img/member/w000800_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,W000779,S,"Wyden, Ron",Democratic,OR,2024-11-04T13:42:34Z,https://api.congress.gov/v3/member/W000779?for...,"<a href=""http://www.senate.gov/artandhistory/h...",https://www.congress.gov/img/member/w000779_20...,D,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,C001087,01,"Crawford, Eric A. ""Rick""",Republican,AR,2024-06-08T18:40:14Z,https://api.congress.gov/v3/member/C001087?for...,Image courtesy of the Member,https://www.congress.gov/img/member/c001087_20...,R,...,None,0.460,0.307,-161.93731,0.85720,1051.0,66.0,None,0.466,0.174
535,S001185,07,"Sewell, Terri A.",Democratic,AL,2024-06-08T18:40:14Z,https://api.congress.gov/v3/member/S001185?for...,Image courtesy of the Member,https://www.congress.gov/img/member/s001185_20...,D,...,None,-0.402,0.396,-51.44477,0.94995,1002.0,18.0,None,-0.524,0.368
536,A000055,04,"Aderholt, Robert B.",Republican,AL,2024-06-08T18:40:14Z,https://api.congress.gov/v3/member/A000055?for...,Image courtesy of the Member,https://www.congress.gov/img/member/a000055_20...,R,...,None,0.405,0.467,-173.56776,0.84683,1044.0,85.0,None,0.464,0.271
537,R000575,03,"Rogers, Mike D.",Republican,AL,2024-06-08T18:40:14Z,https://api.congress.gov/v3/member/R000575?for...,Congressional Pictorial Directory,https://www.congress.gov/img/member/116_rp_al_...,R,...,None,0.379,0.376,-161.36951,0.85666,1043.0,79.0,None,0.449,0.348


In [6]:
r =ct.summarize_news('N000188', engine)

In [8]:
r.json()

{'status': 'ok', 'totalResults': 0, 'articles': []}

In [31]:
news_query = 'Tim Kaine congress'
r = requests.get('https://newsapi.org/v2/everything',
            params={'q': news_query, 'apiKey': ct.newskey, 'searchIn': 'content'},
            headers={'User-Agent': ct.useragent})
r.json()

{'status': 'ok',
 'totalResults': 32,
 'articles': [{'source': {'id': 'breitbart-news', 'name': 'Breitbart News'},
   'author': 'Bradley Jaye, Bradley Jaye',
   'title': 'Tim Kaine Projected to Win Reelection to Senate',
   'description': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\nThe post Tim Kaine Projected to Win Reelection to Senate appeared first on Breitbart.',
   'url': 'https://www.breitbart.com/politics/2024/11/06/tim-kaine-projected-to-win-reelection-to-senate/',
   'urlToImage': 'https://media.breitbart.com/media/2024/11/tim-paine-640x335.jpg',
   'publishedAt': '2024-11-06T08:06:54Z',
   'content': 'Sen. Tim Kaine (D-VA) won reelection in the Commonwealth of Virginia on Tuesday night, defeating Republican challenger Hung Cao.\r\nKaines seat was considered relatively safe, although Republicans were… [+2104 chars]'},
  {'source': {'id': 'new-york-magazine', 'name': 'New York Magazine'},
  

In [36]:
content = [x['title']   for x in r.json()['articles']]

In [37]:
input_text = ' '.join(content)

In [38]:
ct.get_summary_text(input_text)

"Tim Kaine Projected to Win Reelection to Senate Trump Can Prosecute Anybody He Wants, Transition Leader Says Why That Kamala Harris Saturday Night Live Appearance Triggered an FCC Response Links 10/23/2024 Columbia-Class and Virginia-Class Submarines: Overbudget and Behind Schedule (Why?)LIVE: Northern Virginia and statewide election results In Virginia, a Vietnamese American Community Divided Falls Church elected leaders wary of Trump return, GOP dominance in Congress Morning Notes for October 25, 2024 Franks: Dems fumbled by passing over Hillary for Harris Trump’s mainstream picks for top foreign policy posts could reassure allies — and worry China An hour-by-hour guide to watching the races on election night Trump's mainstream picks for top foreign policy posts could reassure allies — and worry China Trump's mainstream picks for top foreign policy posts could reassure allies - and worry China Sen. Tim Kaine wins 3rd term in Washington Rep. Gerry Connolly reveals cancer treatment af